# Load the dataset

In [1]:
import pandas as pd

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df=pd.read_csv("train.csv")

In [4]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Preprocessing

In [5]:
(df.isnull().sum()==0).count()

785

In [6]:
df.shape

(42000, 785)

In [7]:
x=df.drop("label",axis="columns")

In [8]:
y=df["label"]

# Hyperparameter tuning

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
def find_best_model(x,y):
    model_params={
        'LogisticRegression':{
            'model':LogisticRegression(),
            'params':{
                'C': [1,5,10],
                'penalty': ['l1','l2']
            }
        },
        'DecisionTreeClassifier':{
            'model':DecisionTreeClassifier(),
            'params':{
                'criterion':['gini','entropy']
            }
        },
        'SVC':{
            'model':SVC(),
            'params':{
                'C':[1,2,3],
                'kernel':['rbf','linear']
            }
        },
        'RandomForestClassifier':{
            'model':RandomForestClassifier(),
            'params':{
                'n_estimators':[10,20,30],
                'criterion':['gini','entropy']
            }
        },
        'GaussianNB':{
            'model':GaussianNB(),
            'params':{
                
            }
        }
    }
    scores=[]
    cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    for model_name,mp in model_params.items():
        clf=GridSearchCV(mp['model'],mp['params'],cv=cv,return_train_score=False)
        clf.fit(x,y)
        scores.append({
            'model':model_name,
            'best_score':clf.best_score_,
            'best_params':clf.best_params_
        })
    return pd.DataFrame(scores,columns=["model","best_score","best_params"])

In [ ]:
find_best_model(x,y)

# Cross validation

In [10]:
cross_val_score(LogisticRegression(),x,y)

C:\Users\ADITYA\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\ADITYA\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

array([0.91642857, 0.91833333, 0.91309524, 0.91428571, 0.9172619 ])

In [11]:
cross_val_score(DecisionTreeClassifier(),x,y)

array([0.8577381 , 0.85464286, 0.85666667, 0.85      , 0.85297619])

In [12]:
cross_val_score(GaussianNB(),x,y)

array([0.55988095, 0.55714286, 0.55940476, 0.55095238, 0.55892857])

In [13]:
cross_val_score(SVC(),x,y)

array([0.97440476, 0.97630952, 0.97345238, 0.97630952, 0.97714286])

# Model Training

In [ ]:
model=SVC()
model.fit(x,y)

In [ ]:
import pickle
with open("Digits_model.pickle","wb") as f:
    pickle.dump(model,f)